# SVM_and_KNN.ipynb

本 Notebook 將詳細介紹 SVM 與 KNN 兩種經典的傳統機器學習算法。內容將涵蓋：

1. **支持向量機 (SVM)**
   - SVM 分類（線性核、RBF核）
   - 多元分類（Iris 資料集）
   - SVR 回歸範例

2. **K 最近鄰 (KNN)**
   - KNN 分類範例（手寫數字資料集、合成資料）
   - KNN 回歸範例
   - 調整 K 值、距離度量和加權策略

3. 資料前處理、Pipeline、Cross-Validation
4. 評估指標 (Accuracy, Precision, Recall, F1, MSE, R²)

透過本 Notebook，您將了解如何使用 `scikit-learn` 執行 SVM 與 KNN 的各種任務，以及如何針對不同的情境選擇合適的參數和前處理方法。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid", font_scale=1.2)

from sklearn.datasets import load_iris, load_digits, make_classification, make_regression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

## 1. 支持向量機 (SVM)

SVM 是一種強大的分類與回歸方法，透過找尋最大間隔的超平面以分離資料點。對於非線性可分的資料，可透過核函數 (Kernel) 將資料映射至高維空間，使之在該空間線性可分。

### 1.1 SVM 分類 (線性核)

使用合成資料展示二元分類情境，先嘗試線性核的 SVM。

In [ ]:
X_bin, y_bin = make_classification(n_samples=500, n_features=2, n_informative=2, n_redundant=0, random_state=42)
X_train_bin, X_test_bin, y_train_bin, y_test_bin = train_test_split(X_bin, y_bin, test_size=0.3, random_state=42)

svm_linear = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(kernel='linear'))
])
svm_linear.fit(X_train_bin, y_train_bin)
y_pred_lin = svm_linear.predict(X_test_bin)
acc_lin = accuracy_score(y_test_bin, y_pred_lin)
print("線性核SVM分類-合成資料:")
print(f"Accuracy: {acc_lin:.4f}")
print(classification_report(y_test_bin, y_pred_lin))

### 1.2 SVM 分類 (RBF核)

RBF核(SVC(kernel='rbf'))能捕捉非線性邊界，常用於較複雜的資料分佈。

In [ ]:
svm_rbf = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(kernel='rbf', gamma='scale'))
])
svm_rbf.fit(X_train_bin, y_train_bin)
y_pred_rbf = svm_rbf.predict(X_test_bin)
acc_rbf = accuracy_score(y_test_bin, y_pred_rbf)
print("RBF核SVM分類-合成資料:")
print(f"Accuracy: {acc_rbf:.4f}")
print(classification_report(y_test_bin, y_pred_rbf))

### 1.3 Iris 資料集多元分類

SVM可自然擴展至多分類問題(One-vs-One策略)。

In [ ]:
iris = load_iris()
X_iris = iris.data
y_iris = iris.target
X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(X_iris, y_iris, test_size=0.3, random_state=42)

svm_iris = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(kernel='rbf'))
])
svm_iris.fit(X_train_i, y_train_i)
y_pred_i = svm_iris.predict(X_test_i)
acc_i = accuracy_score(y_test_i, y_pred_i)
print("SVM多元分類 (Iris):")
print(f"Accuracy: {acc_i:.4f}")
print(classification_report(y_test_i, y_pred_i, target_names=iris.target_names))

### 1.4 SVM 回歸 (SVR)

SVR用於回歸任務，可使用不同核函數擬合非線性關係。

In [ ]:
X_reg, y_reg = make_regression(n_samples=200, n_features=3, noise=10, random_state=42)
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

svr = Pipeline([
    ('scaler', StandardScaler()),
    ('svr', SVR(kernel='rbf', C=1.0))
])
svr.fit(X_train_r, y_train_r)
y_pred_svr = svr.predict(X_test_r)
mse_svr = mean_squared_error(y_test_r, y_pred_svr)
r2_svr = r2_score(y_test_r, y_pred_svr)
print("SVR回歸 (RBF核):")
print(f"MSE: {mse_svr:.4f}")
print(f"R²: {r2_svr:.4f}")

## 2. K 最近鄰 (KNN)

KNN 通過觀察最近的 K 個鄰居投票(分類)或取平均(回歸)來進行預測。無需訓練過程，但對資料規模敏感。

### 2.1 KNN 分類 (手寫數字資料集)

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()
X_dig = digits.data
y_dig = digits.target

X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_dig, y_dig, test_size=0.3, random_state=42)

knn_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=5))
])
knn_clf.fit(X_train_d, y_train_d)
y_pred_d = knn_clf.predict(X_test_d)
acc_d = accuracy_score(y_test_d, y_pred_d)
print("KNN分類 (Digits):")
print(f"Accuracy: {acc_d:.4f}")
print(classification_report(y_test_d, y_pred_d))

### 2.2 KNN 分類 (合成資料) 調整 K 值
嘗試不同的 K 值，觀察效能變化。

In [ ]:
X_knn, y_knn = make_classification(n_samples=500, n_features=2, n_informative=2, random_state=42)
X_train_k, X_test_k, y_train_k, y_test_k = train_test_split(X_knn, y_knn, test_size=0.3, random_state=42)

acc_values = {}
for k in [1, 3, 5, 11, 21]:
    knn_k = Pipeline([
        ('scaler', StandardScaler()),
        ('knn', KNeighborsClassifier(n_neighbors=k))
    ])
    knn_k.fit(X_train_k, y_train_k)
    y_pred_k = knn_k.predict(X_test_k)
    acc_k = accuracy_score(y_test_k, y_pred_k)
    acc_values[k] = acc_k

print("K值與Accuracy:", acc_values)

### 2.3 KNN 回歸範例


In [ ]:
X_knr, y_knr = make_regression(n_samples=200, n_features=2, noise=20, random_state=42)
X_train_kr, X_test_kr, y_train_kr, y_test_kr = train_test_split(X_knr, y_knr, test_size=0.2, random_state=42)

knn_reg = Pipeline([
    ('scaler', StandardScaler()),
    ('knr', KNeighborsRegressor(n_neighbors=5))
])
knn_reg.fit(X_train_kr, y_train_kr)
y_pred_kr = knn_reg.predict(X_test_kr)
mse_kr = mean_squared_error(y_test_kr, y_pred_kr)
r2_kr = r2_score(y_test_kr, y_pred_kr)
print("KNN回歸:")
print(f"MSE: {mse_kr:.4f}")
print(f"R²: {r2_kr:.4f}")

## Pipeline、Cross-Validation 示範

以 SVM 為例，進行參數搜尋（GridSearchCV）與 Cross-Validation。

In [ ]:
param_grid = {
    'svc__C': [0.1, 1, 10],
    'svc__gamma': ['scale', 0.1, 1]
}

svm_grid = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(kernel='rbf'))
])

X_g, y_g = make_classification(n_samples=300, n_features=4, n_informative=2, random_state=42)
X_train_g, X_test_g, y_train_g, y_test_g = train_test_split(X_g, y_g, test_size=0.3, random_state=42)

grid_search = GridSearchCV(svm_grid, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_g, y_train_g)
print("最佳參數:", grid_search.best_params_)
print("最佳得分:", grid_search.best_score_)

y_pred_grid = grid_search.predict(X_test_g)
print("Test Accuracy:", accuracy_score(y_test_g, y_pred_grid))

## 模型選擇指南與建議

- SVM：
  - 適用於中小型資料集、特徵較多但維度不過高的情況。
  - Non-linear kernel (e.g. RBF) 適合複雜決策邊界。
  - 通常需要調參 (C, gamma) 並執行特徵縮放。

- KNN：
  - 簡單易理解，無需訓練階段。
  - 對距離計算敏感，需特徵縮放。對資料量敏感，資料多時預測變慢。
  - K 值需透過 Cross-Validation 決定，K 太小易過擬合，K 太大模型過於平滑。

對於大規模資料集:
 - SVM可使用序列最小優化（SMO）、近似最近鄰搜索技術加快預測。
 - KNN可透過降維(PCA)或索引結構(KD-Tree, Ball-Tree)加速最近鄰搜索。

評估方法:
 - 分類：Accuracy、Precision、Recall、F1-score、AUC。
 - 回歸：MSE、MAE、R²。

透過 Cross-Validation、GridSearchCV 或 RandomizedSearchCV 找到較佳參數設置，以獲得更好的模型表現。